# Association rule mining using apriori algorithm
Using Lido's transaction data in 2019, find out what items customers tend to order together.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from apyori import apriori

In [2]:
#load csv files as dataframe
menu_items = pd.read_csv("data/menu_items.csv", index_col="Id")
transactions = pd.read_csv("data/transactions.csv", index_col="Id")
cols = ['Date', 'Time', 'DayOfWeek', 'isHoliday', 'Price', 'Quantity', 'MenuItemId', 'TransactionId', 'Store']
transaction_products = pd.read_csv("data/transaction_products.csv", usecols=cols)

In [3]:
#check the data in menu_items
display(menu_items.head())
print()
display(menu_items.info())

,Name,Price,Category,IsActive
Id,,,,
5,Beef Broccoli - Mucho,320.0,12,True
6,Beef Ampalaya - Mucho,295.0,12,True
7,Beef Sotanghon in Hot Pot,295.0,12,True
8,Iced Tea Bottomless,95.0,24,True
9,Lido Iced Coffee,140.0,24,True



<class 'pandas.core.frame.DataFrame'>
Int64Index: 446 entries, 5 to 450
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      446 non-null    object 
 1   Price     446 non-null    float64
 2   Category  446 non-null    int64  
 3   IsActive  446 non-null    bool   
dtypes: bool(1), float64(1), int64(1), object(1)
memory usage: 14.4+ KB


None

In [4]:
#check the data in transactions
display(transactions.head())
print()
display(transactions.info())

,Date,Time,DayOfWeek,isHoliday,Store,Type,HeadCount,Gross
Id,,,,,,,,
42626,2019-01-01,09:03:57.763000,Tuesday,False,13,1,1,2720.0
42627,2019-01-01,09:58:05.643000,Tuesday,False,13,1,1,1365.0
42628,2019-01-01,10:00:36.140000,Tuesday,False,13,2,1,952.0
42629,2019-01-01,10:36:46.447000,Tuesday,False,13,1,1,205.0
42630,2019-01-01,11:03:01.163000,Tuesday,False,13,1,1,1030.0



<class 'pandas.core.frame.DataFrame'>
Int64Index: 292186 entries, 42626 to 344152
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Date       292186 non-null  object 
 1   Time       292186 non-null  object 
 2   DayOfWeek  292186 non-null  object 
 3   isHoliday  292186 non-null  bool   
 4   Store      292186 non-null  int64  
 5   Type       292186 non-null  int64  
 6   HeadCount  292186 non-null  int64  
 7   Gross      292186 non-null  float64
dtypes: bool(1), float64(1), int64(3), object(3)
memory usage: 18.1+ MB


None

In [5]:
#check the data in transaction_pro
display(transaction_products.head())
print()
display(transaction_products.info())

,Date,Time,DayOfWeek,isHoliday,Price,Quantity,MenuItemId,TransactionId,Store
0,2019-01-01,09:04:26.933000,Tuesday,False,255.0,4.0,39,42626,13.0
1,2019-01-01,09:04:26.947000,Tuesday,False,340.0,5.0,91,42626,13.0
2,2019-01-01,09:58:51.460000,Tuesday,False,630.0,1.0,31,42627,13.0
3,2019-01-01,09:58:51.477000,Tuesday,False,215.0,1.0,166,42627,13.0
4,2019-01-01,09:58:51.477000,Tuesday,False,520.0,1.0,188,42627,13.0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050058 entries, 0 to 1050057
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   Date           1050058 non-null  object 
 1   Time           1050058 non-null  object 
 2   DayOfWeek      1050058 non-null  object 
 3   isHoliday      1050058 non-null  bool   
 4   Price          1050058 non-null  float64
 5   Quantity       1050058 non-null  float64
 6   MenuItemId     1050058 non-null  int64  
 7   TransactionId  1050058 non-null  int64  
 8   Store          1050058 non-null  float64
dtypes: bool(1), float64(3), int64(2), object(3)
memory usage: 65.1+ MB


None

In [6]:
#preprocess transaction products data
#only retain relevant columns
data = transaction_products[['MenuItemId','TransactionId']]

#group by transaction id to see which items were ordered together
grouped = data.groupby('TransactionId')

#convert df to a list of list (required input format for using apyori)
transactions = [list(grouped.get_group(k)['MenuItemId']) for k in grouped.groups.keys()]
transactions[:5]

[[39, 91], [31, 166, 188], [104, 167, 91], [132, 118], [37, 36, 130, 102]]

In [7]:
#apply apriori algorithm
'''
[parameters]
min_support=0.0017
 -> items that are purchased at least 5 times a day
    = 5*365/total number of transaction products in 2019
min_confidence=.2
 -> items are purchased together at least 20% of the time
min_lift=3
 -> items are at least 3 times more likely to be bought together
min_length=2
 -> minimum number of items in the rule is 2
'''

association_results = list(apriori(transactions, min_support=.0017, min_confidence=.2, min_lift=3, min_length=2))
association_results[0]

RelationRecord(items=frozenset({10, 11}), support=0.002204083029587419, ordered_statistics=[OrderedStatistic(items_base=frozenset({11}), items_add=frozenset({10}), confidence=0.20827943078913325, lift=3.7443010819616624)])

In [8]:
def get_result_table(results):
    res_dict = {}
    for i, item in enumerate(association_results):
        products = list(item[0])
        base_item = products[0]
        added_item = products[1]
        support = item[1]
        confidence = item[2][0][2]
        lift = item[2][0][3]

        res_dict[i] = {
            'base_item':base_item,
            'added_item':added_item,
            'support':support,
            'confidence':confidence,
            'lift':lift
        }    
    return res_dict

def get_result(results):
    #convert result to df
    res_dict = get_result_table(results)
    res_df = pd.DataFrame.from_dict(res_dict, orient='index')

    #sort by lift column (descending)
    res_df = res_df.sort_values(by='lift', ascending=False, ignore_index=True)

    #replace base_item with actual item names
    res_df = res_df.merge(menu_items['Name'],how='left',left_on='base_item',right_index=True)
    res_df.drop(columns='base_item',inplace=True)
    res_df.rename(columns={'Name':'base_item'},inplace=True)

    #replace added_item with actual item names
    res_df = res_df.merge(menu_items['Name'],how='left',left_on='added_item',right_index=True)
    res_df.drop(columns='added_item',inplace=True)
    res_df.rename(columns={'Name':'added_item'},inplace=True)

    #rearrange the order of columns
    cols = ['base_item','added_item','support','confidence','lift']
    res_df = res_df[cols]

    return res_df

In [9]:
'''
ex. base_item : Kung pao chicken, added_item : mango jello promo
support - probability of "kung pao chicken" to be ordered is 0.3%
confidence - probability of "kung pao chicken" to be ordered when customer orders "mango jello promo" is 24%
lift - customers are 71 times more likely to order "kung pao chicken" when they order "mango jello promo"
=> suggest "kung pao chicken" when customer order "mango jello promo"

**suggest base_item when customer orders base_item
'''
res_df = get_result(association_results)
res_df

,base_item,added_item,support,confidence,lift
0,Kung pao chicken,Mango jello promo,0.003460,0.249199,71.524740
1,BF Daing na Bangus - Garlic Rice,Upgrade: Hot chocolate,0.001800,0.332911,11.501917
2,Upgrade: Hot chocolate,BF Classic Medley - Plain Rice,0.004846,0.326870,11.293184
3,Upgrade: Hot chocolate,BF Beef Tapa - Garlic Rice,0.004076,0.274931,11.158583
4,Iced tea - pitcher,Yangchow Fried Rice - Familia,0.003974,0.280503,10.479304
5,Drunken Lechon Macau - Mucho,Yangchow Fried Rice - Familia,0.001797,0.251316,9.388938
6,Fried Chicken (whole),Yangchow Fried Rice - Familia,0.003823,0.246524,9.209896
7,Iced tea - toppings upgrade (12oz),Spareribs in Salt and Pepper Rice,0.002454,0.265261,9.147319
8,Siopao Asado,Siopao Bola-bola,0.006092,0.291469,9.100536
9,Iced tea - toppings upgrade (12oz),Beef with Tomato and Mushroom Rice,0.002030,0.219386,8.694054


## How to make a use of the results?
- Pick a few most associated items for each product  
-> suggest the associated items when customer orders the product

## Improvements to be made
- add more input data
- when pick the suggestion, consider suggested item's price (put more weight on items with higher price?)
- do different stores have different tendencies? or more ore less the same?
- can I take other elements (like number of customers, age, weather, time, etc) into account to improve the accuracy of suggestion?
- how do the results change if I disregard the size (solo, mucho, familia..)?